# Classification Model - XGBoost

In [111]:
#Importing all libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
import os
from itertools import combinations
from scipy.stats import chi2_contingency
import category_encoders as ce
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.utils.class_weight import compute_sample_weight

In [112]:
print(os.getcwd())

c:\Users\ntama\Documentos\Data Science\Codecademy\projects\Date-A-Scientist - Final Project\OKCupid-Date-A-Scientist-Starter\date-a-scientist-project\notebooks


## Importing the dataset

In [113]:
profiles_processed = pd.read_csv("../data/profiles_processed.csv", index_col=False)
profiles_processed = profiles_processed.loc[:, ~profiles_processed.columns.str.contains('^Unnamed')]
profiles_processed.head()

,age,body_type,drinks,drugs,height,sex,smokes,diet_anything,diet_halal,diet_kosher,diet_other,diet_vegan,diet_vegetarian
0,-1.093889,2,1,0,1.678374,1,1,True,False,False,False,False,False
1,0.281388,1,2,1,0.426738,1,0,False,False,False,True,False,False
2,0.598759,1,1,0,-0.073917,1,0,True,False,False,False,False,False
3,-0.988098,1,1,0,0.677065,1,0,False,False,False,False,False,True
4,-0.353355,0,1,0,-0.574571,1,0,True,False,False,False,False,False


In [114]:
print(profiles_processed.columns)

Index(['age', 'body_type', 'drinks', 'drugs', 'height', 'sex', 'smokes',
       'diet_anything', 'diet_halal', 'diet_kosher', 'diet_other',
       'diet_vegan', 'diet_vegetarian'],
      dtype='object')


## Defining predictors and labels for training and test sets

X = predictors = ['age', 'drinks', 'drugs', 'height', 'sex', 'smokes', 'diet_anything', 'diet_halal', 'diet_kosher', 'diet_other', 'diet_vegan', 'diet_vegetarian']</br>
y = labels = 'body_type'


In [115]:
X = profiles_processed.drop(['body_type'], axis=1)
y = profiles_processed['body_type']

Are classes balanced?

In [116]:
print(profiles_processed['body_type'].value_counts(1))

body_type
0    0.461221
1    0.390788
2    0.147991
Name: proportion, dtype: float64


### Splitting the data into train and test

Since classes are imbalanced, y must be stratified

In [117]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2, stratify=y)
weights = compute_sample_weight(class_weight='balanced', y=y_train)

## Model initialization

In [118]:
model = XGBClassifier(
    n_estimators=500,
    max_depth=3,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    eval_metric='mlogloss'
)

### Training the model

In [119]:
model.fit(X_train, y_train, sample_weight=weights)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=500, n_jobs=None,
              num_parallel_tree=None, ...)

Predictions

In [120]:
y_pred = model.predict(X_test)

Confusion Matrix

In [121]:
print(confusion_matrix(y_test, y_pred))

[[3122  788 1620]
 [1893  804 1988]
 [ 393  180 1201]]


Assessing metrics

In [122]:
print(f"Accuracy Score, % total de predicciones correctas sobre todos los casos: {accuracy_score(y_test, y_pred)}")
print(f"{classification_report(y_test, y_pred)}")

Accuracy Score, % total de predicciones correctas sobre todos los casos: 0.4276420051714071
              precision    recall  f1-score   support

           0       0.58      0.56      0.57      5530
           1       0.45      0.17      0.25      4685
           2       0.25      0.68      0.36      1774

    accuracy                           0.43     11989
   macro avg       0.43      0.47      0.39     11989
weighted avg       0.48      0.43      0.41     11989



In [123]:
model.feature_importances_


array([0.02976763, 0.02950939, 0.02276517, 0.03746576, 0.7055662 ,
       0.05097557, 0.02093664, 0.01548158, 0.01493841, 0.02411357,
       0.01843689, 0.03004309], dtype=float32)